In [1]:
# all the emails are from the webstie form
email_text_1 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:48
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: Neue Nachricht von rubitherm.eu - Anfrage für Probentransport bei 38°C +-1 °C

 

 

 

Best greetings / Mit freundlichen Grüßen

--------------------------------------------------

 

Anne Neumeyer

 

Rubitherm Technologies GmbH

Imhoffweg 6

D-12307 Berlin

 

Geschäftsführer: Lutz Klinkner, Thomas Braun

Amtsgericht: Berlin-Charlottenburg HRB 86322B

 

Tel: +49 30 71 09 622-0             e-mail: anne.neumeyer@rubitherm.com

Fax: +49 30 71 09 622-22          http://www.rubitherm.com

 

Von: Matthias Moser <info@rubitherm.com>
Gesendet: Sonntag, 24. August 2025 14:38
An: info@rubitherm.com
Betreff: Neue Nachricht von rubitherm.eu - Anfrage für Probentransport bei 38°C +-1 °C

 

Country: Deutschland
Phone: 0491721442009
Company: Moser Konstruktion GmbH &
Name: Matthias Moser
E-Mail: m.moser@moser-konstruktion.de

Sehr geehrte Damen und Herren,

wir haben eine Anwendung im Prototypenbau für den Transport von Blutproben, welche bei 38°C +-1°C transportiert werden müssen.

hierzu habe ich RT38 angeschaut. das wäre meiner Meinung nach perfekt.

ich benötigte noch eine technische Beratung und Informationen zum möglichen Aufbau des Systems.

es sollen 4 Proben in sogenannten Monovetten transportiert und gelagert werden. Das System muss für mindestens 8 Stunden die Temperatur halten. Es kann nachgeheizt werden. mir ist noch nicht klar, in welchen Formen und Möglichkeiten wir RT38 verbauen bzw. verwenden können.

bitte um Kontaktaufnahme

ich danke Ihnen

Mit freundlichen Grüßen Matthias Moser"""
email_text_2 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:47
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: Neue Nachricht von rubitherm.eu - sample enquiry for RT11HC or RT12

 

 

 

Best greetings / Mit freundlichen Grüßen

--------------------------------------------------

 

Anne Neumeyer

 

Rubitherm Technologies GmbH

Imhoffweg 6

D-12307 Berlin

 

Geschäftsführer: Lutz Klinkner, Thomas Braun

Amtsgericht: Berlin-Charlottenburg HRB 86322B

 

Tel: +49 30 71 09 622-0             e-mail: anne.neumeyer@rubitherm.com

Fax: +49 30 71 09 622-22          http://www.rubitherm.com

 

Von: Jesper Godrim Jensen <info@rubitherm.com>
Gesendet: Donnerstag, 21. August 2025 10:48
An: info@rubitherm.com
Betreff: Neue Nachricht von rubitherm.eu - sample enquiry for RT11HC or RT12

 

Country: Denmark
Phone: 28777689
Company: grader
Name: Jesper Godrim Jensen
E-Mail: Godrimjensen@gmail.com

Dear Sir or Madam,

I am interested in your products RT11HC and RT12 and would like to kindly ask whether it is possible to receive samples for evaluation. We are currently exploring suitable phase change materials, and these two products appear particularly relevant to our needs.

In addition, could you please provide the corresponding Technical Data Sheets (TDS) and Safety Data Sheets (SDS) for these products?

I would appreciate any information on availability, sample sizes, and conditions.

Thank you in advance for your support.

Kind regards,
Jesper G. Jensen"""

email_text_3 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:51
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: Neue Nachricht von rubitherm.eu - Product enquiry

 

 

 

Best greetings / Mit freundlichen Grüßen

--------------------------------------------------

 

Anne Neumeyer

 

Rubitherm Technologies GmbH

Imhoffweg 6

D-12307 Berlin

 

Geschäftsführer: Lutz Klinkner, Thomas Braun

Amtsgericht: Berlin-Charlottenburg HRB 86322B

 

Tel: +49 30 71 09 622-0             e-mail: anne.neumeyer@rubitherm.com

Fax: +49 30 71 09 622-22          http://www.rubitherm.com

 

Von: Genevieve Kerr <info@rubitherm.com>
Gesendet: Montag, 11. August 2025 06:29
An: info@rubitherm.com
Betreff: Neue Nachricht von rubitherm.eu - Product enquiry

 

Country: Australia
Phone:
Company: BioCifer
Name: Genevieve Kerr
E-Mail: genevieve.kerr@biocifer.com

Hi, I am looking at purchasing some RT44HC, could I please get a quote for purchasing and shipping to Australia. We would be looking at purchasing around 500g but price depending we may get more or less."""
email_text_4 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:51
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: Neue Nachricht von rubitherm.eu - Anfrage Muster RT54HC für textile Wärmeanwendung

 

 

 

Best greetings / Mit freundlichen Grüßen

--------------------------------------------------

 

Anne Neumeyer

 

Rubitherm Technologies GmbH

Imhoffweg 6

D-12307 Berlin

 

Geschäftsführer: Lutz Klinkner, Thomas Braun

Amtsgericht: Berlin-Charlottenburg HRB 86322B

 

Tel: +49 30 71 09 622-0             e-mail: anne.neumeyer@rubitherm.com

Fax: +49 30 71 09 622-22          http://www.rubitherm.com

 

Von: Nadine Saitner <info@rubitherm.com>
Gesendet: Donnerstag, 7. August 2025 15:05
An: info@rubitherm.com
Betreff: Neue Nachricht von rubitherm.eu - Anfrage Muster RT54HC für textile Wärmeanwendung

 

Country: Germany
Phone:
Company: Student at Nova Business School
Name: Nadine Saitner
E-Mail: saitnernadine@gmail.com

Sehr geehrtes Rubitherm-Team,

ich recherchiere aktuell für ein Produktkonzept im Konsumgüterbereich, bei dem ein PCM-Material mit einem Schmelzpunkt um ca. 54 °C im Zentrum steht. Ihr Produkt RT54HC scheint dafür grundsätzlich sehr gut geeignet.

Könnten Sie mir bitte mitteilen: • ob Sie eine kleinere Menge des Materials (z. B. ca. 300–500 g) zur Verfügung stellen können, idealerweise in mikroverkapselter Form, • oder ob Sie alternativ vorgefertigte Pouches/Pads mit RT54HC anbieten, die sich für eine textile Weiterverarbeitung und Mikrowellenanwendung eignen?

Die Muster würden ausschließlich für Machbarkeits- und Haptiktests im Rahmen eines frühen Prototyps verwendet.

Ich freue mich über Ihre Rückmeldung zu Machbarkeit, Mindestmengen, Preisen und ggf. Versand.

Vielen Dank im Voraus und freundliche Grüße Nadine"""

In [2]:
# direct customer email
email_text_5 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:47
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: Inquiry for PCM Solution at –32 °C

 

 

 

Best greetings / Mit freundlichen Grüßen

--------------------------------------------------

 

Anne Neumeyer

 

Rubitherm Technologies GmbH

Imhoffweg 6

D-12307 Berlin

 

Geschäftsführer: Lutz Klinkner, Thomas Braun

Amtsgericht: Berlin-Charlottenburg HRB 86322B

 

Tel: +49 30 71 09 622-0             e-mail: anne.neumeyer@rubitherm.com

Fax: +49 30 71 09 622-22          http://www.rubitherm.com

 

Von: Turo Tiililä <turo.tiilila@testmill.fi>
Gesendet: Samstag, 23. August 2025 12:50
An: info@rubitherm.com
Betreff: Inquiry for PCM Solution at –32 °C

 

Hello,

We operate a climatic test hall (5 × 5 × 25 m) currently equipped with a cooling system rated at approximately 50 kW.

Some of our customers now require a test where a vehicle is run at elevated idle for 2 hours. We estimate that this will generate an additional heat load of about 60–65 kW in the hall. This results in a surplus of roughly 20 kW for the duration of the 2-hour test (≈40 kWh total) that our existing system cannot compensate for.

The test temperature is –32 °C. Would you have a PCM-based solution capable of handling this additional load?

I would appreciate further information and a quotation for such a PCM system.

Thank you in advance.

Best regards,

Turo Tiililä

Chairman of the Board

Oy Testmill Ltd

+358400435008

turo.tiilila@testmill.fi"""

email_text_6 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:47
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: NEW PCM PO URGENT - TEMPACK
Priorität: Hoch

 

 

 

Best greetings / Mit freundlichen Grüßen

--------------------------------------------------

 

Anne Neumeyer

 

Rubitherm Technologies GmbH

Imhoffweg 6

D-12307 Berlin

 

Geschäftsführer: Lutz Klinkner, Thomas Braun

Amtsgericht: Berlin-Charlottenburg HRB 86322B

 

Tel: +49 30 71 09 622-0             e-mail: anne.neumeyer@rubitherm.com

Fax: +49 30 71 09 622-22          http://www.rubitherm.com

 

Von: Otoniel Guardado <Oguardado@csafeglobal.com>
Gesendet: Donnerstag, 21. August 2025 07:44
An: info@rubitherm.com; esther.kieseritzky@rubitherm.com
Cc: Xavier Diaz <xdiaz@csafeglobal.com>
Betreff: NEW PCM PO URGENT - TEMPACK
Priorität: Hoch

 

Hi Anne/Esther,

 

We want to purchase 1500 kg of PCM asap, can you please confirm us if you have it?

I will send you the PO and schedule the pick up.

 

Regards.

 

Otoniel Guardado

​​​​

Planning and Purchasing Supervisor

CSafe

Confidentiality Notice
This electronic transmission (and/or the documents accompanying it) may contain confidential information. The information is intended only for the use of the individual or entity to which it is addressed. If you are not the intended recipient, you hereby are notified that any disclosure, copying, distribution, electronic storing or the taking of any action in reliance on the contents of this information is strictly prohibited. If you have received this transmission in error, please immediately notify the Sender by reply e-mail and immediately delete this record from your computer system."""

email_text_7 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:54
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: Inquiry on Rubitherm RT 42 PCM Supply in Malaysia

 

(wurde an unseren Partner Fovence in Singapur weitergeleitet)

 

LG
Anne

 

Von: NUR AAINAA SYAHIRAH BINTI RAMLI (MPOB) <aainaa.syahirah@mpob.gov.my>
Gesendet: Mittwoch, 30. Juli 2025 09:47
An: info@rubitherm.com
Betreff: Inquiry on Rubitherm RT 42 PCM Supply in Malaysia

 

Dear Rubitherm Sales Team,

  

I am writing to inquire about the availability and supply options for your RT 42 in Malaysia.
Are there any authorized Malaysian distributors or official resellers?
We are currently planning to use this material for a research and development project related to thermal energy storage.

 

Thank you

 

--

Nur Aainaa Syahirah Ramli (PhD)

Research Officer

Quality and Environmental Assessment Unit

Advanced Oleochemical Technology Division

Malaysian Palm Oil Board

 

Peringatan 

Pengguna Perkhidmatan MyGovUC adalah bertanggungjawab melindungi kerahsiaan data/maklumat Rahsia Rasmi Kerajaan. Adalah diingatkan agar pengguna sentiasa peka dengan SEMUA peraturan, arahan keselamatan dan pekeliling semasa yang berkuatkuasa bagi semua pengendalian data/maklumat Rahsia Rasmi Kerajaan yang berkaitan."""
email_text_8 = """Von: info@rubitherm.com <info@rubitherm.com>
Gesendet: Mittwoch, 27. August 2025 08:54
An: Günzel, Matthias <matthias.guenzel@rubitherm.com>
Betreff: WG: PCM panels

 

 

 

Best greetings / Mit freundlichen Grüßen

--------------------------------------------------

 

Anne Neumeyer

 

Rubitherm Technologies GmbH

Imhoffweg 6

D-12307 Berlin

 

Geschäftsführer: Lutz Klinkner, Thomas Braun

Amtsgericht: Berlin-Charlottenburg HRB 86322B

 

Tel: +49 30 71 09 622-0             e-mail: anne.neumeyer@rubitherm.com

Fax: +49 30 71 09 622-22          http://www.rubitherm.com

 

Von: Rooi Colchester - Bath Canal Craft Ltd. <bathcanalcraft@icloud.com>
Gesendet: Dienstag, 29. Juli 2025 15:05
An: info@rubitherm.com
Betreff: PCM panels

 

Hi, 

 

I’m looking to use a PCM panel in my narrowboats I build. I think a material changing phase at about 24-26ºC would be about right, as the concept is to provide some protection against peak temperatures during heat waves. 

 

The primary application would be to get the material in the ceiling under the roof, which has a small curvature, about 100mm over 1.4 meters. 

 

It would also be useful if I could fit some panels to the side panels (vertical sides) of the boat, so I’m wondering if you have a micro-encapsulation that would prevent the PCM material from pooling when liquid?

 

Please can you advise if you sell a suitable product, and send some info on any suitable products you may have? 


I’m based in the UK.

 

Many thanks,


Rooi Colchester
Bath Canal Craft Ltd. 

T: 07538784613
W: https://bathcanalcraft.co.uk
E: bathcanalcraft@icloud.com"""

In [23]:
emails = {
    "email_1": email_text_1,
    "email_2": email_text_2,
    "email_3": email_text_3,
    "email_4": email_text_4,
    "email_5": email_text_5,
    "email_6": email_text_6,
    "email_7": email_text_7,
    "email_8": email_text_8,
}

for key, text in emails.items():
    output = extract_email_data(text)
    print(f"\n--- {key} ---")
    print(json.dumps(output, indent=4, ensure_ascii=False))


--- email_1 ---
{
    "extracted_data": {
        "first_name": "Matthias",
        "last_name": "Moser",
        "company": [
            "Moser Konstruktion GmbH &"
        ],
        "customer_phone": "0491721442009",
        "email": [
            "m.moser@moser-konstruktion.de"
        ],
        "roles": null,
        "address": "Deutschland",
        "website": null,
        "message": "Sehr geehrte Damen und Herren,\n\nwir haben eine Anwendung im Prototypenbau für den Transport von Blutproben, welche bei 38°C +-1°C transportiert werden müssen.\n\nhierzu habe ich RT38 angeschaut. das wäre meiner Meinung nach perfekt.\n\nich benötigte noch eine technische Beratung und Informationen zum möglichen Aufbau des Systems.\n\nes sollen 4 Proben in sogenannten Monovetten transportiert und gelagert werden. Das System muss für mindestens 8 Stunden die Temperatur halten. Es kann nachgeheizt werden. mir ist noch nicht klar, in welchen Formen und Möglichkeiten wir RT38 verbauen bzw. verwenden

In [22]:
# pip install SpaCy
# python -m spacy download en_core_web_sm
import re
import json
import spacy
from spacy.lang.en import English

# Load spaCy (en_core_web_sm)
nlp = spacy.load("en_core_web_sm")

# Regex to find the start of the signature block
SIGN_OFF_REGEX = re.compile(
    r"(?:Mit freundlichen Grüßen|Freundliche Grüße|Beste Grüße|Viele Grüße|Herzliche Grüße|"
    r"Liebe Grüße|Schöne Grüße|Grüße|Best regards|Kind regards|Regards|Sincerely|"
    r"Yours sincerely|Yours faithfully|Thank you|Thanks)",
    re.IGNORECASE
)

# Regex to find email headers to determine customer block
HEADER_REGEX = re.compile(
    r"^\s*(Von:)",
    re.IGNORECASE | re.MULTILINE
)

# List of predefined tags
TAGS_LIST = [
    "heiz", "ish 2025", "kälte", "wärme", "kühlh", "lüftung", "tga",
    "messe", "medi", "pcm", "pharma", "logistik", "plan", "wett",
    "uni", "trak", "spei"
]

# List of generic email domains
GENERIC_DOMAINS = [
    "gmail.com", "outlook.com", "yahoo.com", "hotmail.com", "icloud.com",
    "web.de", "posteo.de", "googlemail.com", "live.com", "aol.com",
    "msn.com", "mail.ru"
]
# -------------------------------
# Signature Block finder
# -------------------------------
def get_signature_block(lines: list) -> list:
    """
    Scans lines from the bottom to find the signature block.
    """
    for i in range(len(lines) - 1, -1, -1):
        if SIGN_OFF_REGEX.search(lines[i]):
            return lines[i+1:]
    return lines[-10:]

# -------------------------------
# Tag Extractor
# -------------------------------
def extract_tags(text: str) -> list:
    """
    Extracts predefined tags from the given text.
    """
    found_tags = set()
    text_lower = text.lower()
    for tag in TAGS_LIST:
        if tag in text_lower:
            found_tags.add(tag.title())
    return list(found_tags)

# -------------------------------
# Form Inquiry Extractor
# -------------------------------
def form_inquiry_extractor(text: str) -> dict:
    data = {}
    name_match = re.search(r"Name:\s*(.*)", text)
    if name_match:
        name_parts = name_match.group(1).strip().split()
        if len(name_parts) >= 2:
            data["first_name"] = name_parts[0]
            data["last_name"] = " ".join(name_parts[1:])
    email_match = re.search(r"E-?Mail:\s*([\w\.-]+@[\w\.-]+)", text)
    if email_match:
        data["email"] = email_match.group(1)
    company_match = re.search(r"Company:\s*(.*)", text)
    if company_match:
        data["company"] = company_match.group(1).strip()
    phone_match = re.search(r"Phone:\s*([\d\+][\d\s]+)", text)
    if phone_match:
        data["customer_phone"] = re.sub(r"\s+", "", phone_match.group(1))
    country_match = re.search(r"Country:\s*(.*)", text)
    if country_match:
        data["country"] = country_match.group(1).strip()
    
    data["message"] = None
    if email_match:
        tail = text[email_match.end():]
        tail = tail.lstrip()
        
        stop_patterns = [
            r"\bVon:\b", r"\bGesendet:\b",
            r"\bAn:\b", r"\bBetreff:\b", r"\bFrom:\b", r"\bSent:\b", r"\bTo:\b", r"\bSubject:\b",
            r"(?:Mit freundlichen Grüßen|Freundliche Grüße|Beste Grüße|Viele Grüße|Herzliche Grüße|"
            r"Liebe Grüße|Schöne Grüße|Grüße|Best greetings|Best regards|Kind regards|Regards|Sincerely|"
            r"Yours sincerely|Yours faithfully|Thank you|Thanks)"
        ]
        
        first_cut = len(tail)
        for p in stop_patterns:
            m = re.search(p, tail, re.IGNORECASE)
            if m:
                first_cut = min(first_cut, m.start())

        candidate = tail[:first_cut].strip()
        
        salutation_re = re.compile(r"(?m)^\s*(Sehr|Dear|Hello|Hi|Guten|Good|Kind)\b", re.IGNORECASE)
        s = salutation_re.search(candidate)
        if s:
            data["message"] = candidate[s.start():].strip()
        else:
            if candidate:
                data["message"] = candidate
            else:
                data["message"] = tail.strip() if tail.strip() else None

    if "country" in data:
        data["address"] = data.pop("country")
    
    return {
        "extracted_data": {
            "first_name": data.get("first_name"),
            "last_name": data.get("last_name"),
            "company": data.get("company"),
            "customer_phone": data.get("customer_phone"),
            "email": data.get("email"),
            "roles": None,
            "address": data.get("address"),
            "website": None,
            "message": data.get("message"),
            "tags": [],
        },
        "extracted_by": "form_inquiry_extractor",
    }

# -------------------------------
# Direct Email Extractor
# -------------------------------
def direct_email_extractor(text: str) -> dict:
    """
    Extracts contact information from direct emails based on the provided logic.
    """
    first_name, last_name, email = None, None, None
    companies = []
    
    # Step 1: Extract Name, Email, and Company from 'Von:' line (Highest priority)
    von_line_match = re.search(r"^\s*Von:\s*(?P<name_part>.+?)?\s*<(?P<email>[^>]+)>", text, re.IGNORECASE | re.MULTILINE)
    
    if von_line_match:
        email = von_line_match.group("email").strip()
        
        # Split name from "Von: Name <email>"
        name_part = von_line_match.group("name_part")
        if name_part:
            # Clean up the name part and split into first/last name
            clean_name_part = re.sub(r"\(.*\)|['\"]| - .*", "", name_part).strip()
            name_parts = clean_name_part.split()
            if len(name_parts) >= 2:
                first_name = name_parts[0]
                last_name = " ".join(name_parts[1:])
            else:
                # If only one word in name, assume it's a first name
                if name_parts:
                    first_name = name_parts[0]

        # Extract company from Von: line, but only if not generic
        domain = email.split('@')[-1]
        if domain.lower() not in [d.lower() for d in GENERIC_DOMAINS] and "rubitherm" not in domain.lower():
            companies.append(domain.split('.')[0].replace('-', ' ').replace('_', ' ').title())
            # For specific cases like mpob.gov.my
            if 'gov.my' in domain.lower():
                companies = ['MPOB']


    # Step 2: Find signature block for other info
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    signature_lines = get_signature_block(lines)
    signature_text = "\n".join(signature_lines)

    # Step 3: Regex-based extraction (for clear patterns)
    emails_from_sig = re.findall(r"[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,}", signature_text)
    phones = re.findall(r"(?:\+?\d[\d\s\-\(\)]{6,}\d)", signature_text)
    phones = [re.sub(r"\s{2,}", " ", p).strip() for p in phones]
    websites = re.findall(r"(?:https?://[^\s]+|www\.[^\s]+)", signature_text)

    # Step 4: Fallback company name from signature block if not found in Von line
    if not companies:
        company_regex = re.compile(
            r"\b(?:GmbH|Ltd|Inc|Corp|LLC|LLP|Co\.|S\.A\.|S\.p\.A\.|Pty|PLC)\b",
            re.IGNORECASE
        )
        for line in signature_lines:
            if company_regex.search(line):
                companies.append(line.strip())
                break

    # Step 5: Extract Roles and Address from signature block
    roles = []
    role_keywords = ['manager', 'director', 'supervisor', 'officer', 'head', 'lead',
                     'coordinator', 'specialist', 'consultant', 'planning', 'purchasing', 'engineer']
    for line in signature_lines:
        line_lower = line.lower()
        if any(role_word in line_lower for role_word in role_keywords):
            roles.append(line.strip())
    
    # Address extraction using spaCy and then a regex fallback
    company_addresses = []
    doc = nlp(signature_text)
    for ent in doc.ents:
        if ent.label_ in ("GPE", "LOC"):
            if re.search(r"\d", ent.text) or len(ent.text.split()) > 1:
                company_addresses.append(ent.text.strip())
    
    # Address Regex Fallback (if spaCy finds nothing)
    if not company_addresses:
        address_regex_fallback = re.compile(
            r"\b[a-zA-Z\s]{8,},?\s*\d{1,}\b"
        )
        for line in signature_lines:
            if address_regex_fallback.search(line) and len(line.strip()) <= 15:
                company_addresses.append(line.strip())

    # Final email list consolidation
    final_emails = []
    if email:
        final_emails.append(email)
    
    for e in emails_from_sig:
        if "rubitherm" not in e.lower() and e not in final_emails:
            final_emails.append(e)
    
    # Clean up phone numbers and other single-value lists
    customer_phone = list(set(phones))
    if not customer_phone:
        customer_phone = None
    elif len(customer_phone) == 1:
        customer_phone = customer_phone[0]

    return {
        "extracted_data": {
            "first_name": first_name,
            "last_name": last_name,
            "company": list(set(companies)),
            "customer_phone": customer_phone,
            "email": list(set(final_emails)),
            "roles": list(set(roles)),
            "address": list(set(company_addresses)),
            "website": list(set(websites)),
            "tags": [],
        },
        "extracted_by": "direct_email_extractor",
    }

# -------------------------------
# Dispatcher
# -------------------------------
def extract_email_data(text: str) -> dict:
    """
    Main function to extract data from an email by first identifying the customer's block
    and then dispatching to the appropriate extractor.
    """
    # Step 1: Find the start of the customer's email block (bottom-up)
    lines = text.splitlines()
    customer_block_start = 0
    for i in range(len(lines) - 1, -1, -1):
        if HEADER_REGEX.search(lines[i]):
            customer_block_start = i
            break
    
    customer_block_text = "\n".join(lines[customer_block_start:])

    # Step 2: Extract tags from the customer block
    tags = extract_tags(customer_block_text)

    # Step 3: Dispatch to the correct extractor based on form-like patterns
    if "Name:" in customer_block_text and "Company:" in customer_block_text and "E-Mail:" in customer_block_text:
        result = form_inquiry_extractor(customer_block_text)
    else:
        result = direct_email_extractor(customer_block_text)

    # Step 4: Add tags to the final result and ensure all fields exist
    result["extracted_data"]["tags"] = tags
    
    # Ensure all required keys exist in the final output
    required_keys = ["first_name", "last_name", "company", "customer_phone", "email", "roles", "address", "website"]
    if result["extracted_by"] == "form_inquiry_extractor":
        for key in required_keys:
            if key not in result["extracted_data"]:
                result["extracted_data"][key] = None
    
    if isinstance(result["extracted_data"].get("company"), str):
        result["extracted_data"]["company"] = [result["extracted_data"]["company"]]
    if isinstance(result["extracted_data"].get("email"), str):
        result["extracted_data"]["email"] = [result["extracted_data"]["email"]]
    
    return result